# Import Packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
user_test_folder = volatility_folder + "user_test/"
model_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/FB Prophet/"

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pickle

import torch.nn as nn
import os.path as osp
import torch.utils.data as data
import torch.nn.functional as F

from tqdm import tqdm
import time, matplotlib, torch, os
from statsmodels.tsa.arima_model import ARIMA
from torch.utils.tensorboard import SummaryWriter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
!pip install stockstats

In [5]:
import os
from os import walk
from math import floor
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

import plotly
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import pandas_datareader.data as web
from stockstats import StockDataFrame
#import seaborn as sns

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [7]:
#For DNN / CNN+LSTM
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Flatten, Dense

In [8]:
!pip install -q stockstats

# Data Loading

In [9]:


#Setting the index values as timestamp
def process(code):
  print("Enter the code of the model that you want to implement")
  #code = input()
  #final_code = '^'+code
  final_code = code
  start_date = "2012-01-01"
  end_date = "2021-12-31"
  stock = web.DataReader(final_code , 'stooq',start=start_date, end=end_date)
  stock.columns = ["open","high","low","close","volume"]
  
  # Use online package to generate additional features
  x = StockDataFrame(stock)
  stock = x[['open','high','low','close','volume',
          'boll', 'boll_ub', 'boll_lb',
          'macd', 'macdh', 'macds',
          'rsi_11', 'rsi_14', 'rsi_21']]
  stock.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in stock.index]
  stock = stock.sort_index()

  return stock

In [10]:
stock = process("IBM")

Enter the code of the model that you want to implement


In [11]:
stock

,open,high,low,close,volume,boll,boll_ub,boll_lb,macd,macdh,macds,rsi_11,rsi_14,rsi_21
20120103,128.88,130.23,128.38,128.58,8180058,128.1790,134.932612,121.425388,-1.811274,0.489102,-2.300376,51.281366,47.190995,42.920670
20120104,128.10,128.60,127.63,128.07,6297526,128.3965,135.476771,121.316229,-2.091832,0.330819,-2.422652,48.488535,44.808561,41.192557
20120105,127.56,127.71,126.38,127.46,6466200,128.6025,135.879775,121.325225,-2.367574,0.137783,-2.505356,45.064237,41.897279,39.091982
20120106,127.28,127.34,125.83,125.98,7094969,128.9115,136.502568,121.320432,-2.622973,-0.083171,-2.539802,35.625325,34.061158,33.612565
20120109,125.76,125.80,124.42,125.33,7535572,129.2665,136.943913,121.589087,-2.760113,-0.241104,-2.519009,30.883945,30.223364,31.016798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,129.08,130.08,128.40,130.05,4346229,131.4500,133.282321,129.617679,-0.099397,-0.058552,-0.040846,8.180361,8.545592,8.997984
20211228,130.03,131.63,130.02,131.05,3488601,131.8000,132.899939,130.700061,-0.042801,-0.026749,-0.016052,14.491914,14.911940,15.424720
20211229,130.82,132.61,130.73,131.76,4291007,132.0500,132.611070,131.488930,-0.010219,-0.007870,-0.002349,28.868360,29.305681,29.832936
20211230,132.16,132.77,131.74,132.32,3196152,132.1950,132.548553,131.841447,0.005609,0.002493,0.003116,100.000000,100.000000,100.000000


# FB Prophet Training

## Split the train and test data

In [17]:

def custom_split(data,start,end):
    train = (data.index >= start) & (data.index <= end)
    train_X = data[train]
    
    return train_X

In [18]:
train_X = custom_split(stock,start = 20130101,end = 20171231)
test_X = custom_split(stock,start = 20180101,end = 20181231)

## Reshape the data

In [19]:
train_X = train_X.reset_index()
test_X = test_X.reset_index()

train_X = train_X.rename(columns={"index":"Date"})
test_X = test_X.rename(columns={"index":"Date"})


In [20]:
train_X["Date"]= pd.to_datetime(train_X["Date"], format='%Y%m%d')
test_X["Date"] = pd.to_datetime(test_X["Date"], format='%Y%m%d')

In [21]:
train_X = train_X[["Date","close"]]
train_X = train_X.rename(columns={"Date":"ds","close":"y"})
test_X = test_X[["Date","close"]]
test_X = test_X.rename(columns={"Date":"ds","close":"y"})

# Model Training



In [22]:

prophet_model = Prophet(daily_seasonality = True) 
prophet_model.fit(train_X)


# Model Testing

In [23]:
future = prophet_model.make_future_dataframe(periods=int(len(test_X)*1.5))
forecast = prophet_model.predict(future)


In [24]:
#price_pred = forecast[-len(test_X):][["ds","yhat"]]
price_pred = forecast[["ds","yhat"]]
price_pred = price_pred.rename(columns={"ds":"Date","yhat":"Prediction"})


In [25]:
price_pred

,Date,Prediction
0,2013-01-02,140.549874
1,2013-01-03,140.700033
2,2013-01-04,140.518257
3,2013-01-07,140.470361
4,2013-01-08,140.313265
...,...,...
1630,2019-01-05,110.129915
1631,2019-01-06,110.035326
1632,2019-01-07,110.426021
1633,2019-01-08,110.219035


In [26]:
x = price_pred[-int(len(test_X)*1.5):]
y = test_X.rename(columns={"ds":"Date"})

In [27]:
x

,Date,Prediction
1259,2017-12-30,121.378114
1260,2017-12-31,121.361572
1261,2018-01-01,121.827154
1262,2018-01-02,121.690655
1263,2018-01-03,121.635696
...,...,...
1630,2019-01-05,110.129915
1631,2019-01-06,110.035326
1632,2019-01-07,110.426021
1633,2019-01-08,110.219035


In [28]:
y

,Date,y
0,2018-01-02,125.880
1,2018-01-03,129.330
2,2018-01-04,131.960
3,2018-01-05,132.610
4,2018-01-08,133.410
...,...,...
246,2018-12-24,91.763
247,2018-12-26,95.019
248,2018-12-27,97.060
249,2018-12-28,96.419


In [29]:
final_ = pd.merge(x,y, how='inner', on='Date')
new_test_y = final_[["y"]]
new_test_y.index = final_["Date"]
new_test_y.columns = ["Open"]
fb_pred_y = final_[["Prediction"]]
fb_pred_y.index = final_["Date"]


In [30]:
new_test_y

,Open
Date,
2018-01-02,125.880
2018-01-03,129.330
2018-01-04,131.960
2018-01-05,132.610
2018-01-08,133.410
...,...
2018-12-24,91.763
2018-12-26,95.019
2018-12-27,97.060


In [31]:

fb_pred_y

,Prediction
Date,
2018-01-02,121.690655
2018-01-03,121.635696
2018-01-04,121.721439
2018-01-05,121.475793
2018-01-08,121.241097
...,...
2018-12-24,110.725023
2018-12-26,110.746702
2018-12-27,110.944124


In [32]:
#Performance evaluation
print('FB Prophet PERFORMANCE')
print('r2 score: '+str(r2_score(new_test_y, fb_pred_y)))
print('RMSE : '+str(np.sqrt(mean_squared_error(new_test_y, fb_pred_y))))
print("Mean Absolute Error : " + str(mean_absolute_error(new_test_y,fb_pred_y)))

FB Prophet PERFORMANCE
r2 score: 0.19141116494509347
RMSE : 9.11569749797112
Mean Absolute Error : 7.649421578244


# Model Export

In [33]:
with open(model_folder+"fbprophet.pickle", "wb") as f:
    pickle.dump(prophet_model, f)